In [ ]:
#Importing the Libraries

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
class DecisionStump:
    def __init__(self):
        self.polarity = 1      # The direction of the inequality (1 or -1)
        self.feature_idx = None # Index of the feature to split on
        self.threshold = None   # Threshold value for the split
        self.min_error = float('inf') # The minimum weighted error for this stump

    def fit(self, X, y, sample_weights):
        """
        Finds the best feature and threshold to split the data,
        based on the minimum weighted error.
        """
        n_samples, n_features = X.shape

        # Iterate over every feature
        for feature_i in range(n_features):
            feature_values = X[:, feature_i]
            unique_values = np.unique(feature_values)

            # Iterate over every unique value as a potential threshold
            for threshold in unique_values:
                # Try both polarities
                for p in [1, -1]:
                    predictions = np.ones(n_samples)

                    if p == 1:
                        # If feature value < threshold, predict -1
                        predictions[feature_values < threshold] = -1
                    else:
                        # If feature value > threshold, predict -1
                        predictions[feature_values > threshold] = -1

                    # 4. Calculate the weighted error
                    weighted_error = np.sum(sample_weights[predictions != y])

                    # 5. Check if this is the best stump so far
                    if weighted_error < self.min_error:
                        self.min_error = weighted_error
                        self.polarity = p
                        self.threshold = threshold
                        self.feature_idx = feature_i

    def predict(self, X):
        """
        Makes predictions for a new set of data X.
        """
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)
        feature_values = X[:, self.feature_idx]

        # Classify based on the stored best threshold and polarity
        if self.polarity == 1:
            predictions[feature_values < self.threshold] = -1
        else:
            predictions[feature_values > self.threshold] = -1

        return predictions

In [ ]:

#-----------------------------------------------------------------
# 2. The AdaBoost Classifier
#-----------------------------------------------------------------

class AdaBoost:
    def __init__(self, n_estimators=5):
        self.n_estimators = n_estimators # Number of weak learners (stumps) to train
        self.clfs = []         # List to store the weak learners
        self.alphas = []       # List to store the "amount of say" (alpha) for each learner

    def fit(self, X, y):
        """
        Trains the AdaBoost model.
        """
        n_samples, n_features = X.shape

        # 1. Initialize sample weights equally
        w = np.full(n_samples, (1 / n_samples))

        self.clfs = []
        self.alphas = []

        # 2. Iterate for n_estimators (M in the article)
        for _ in range(self.n_estimators):

            # 3. Train a weak learner (stump) on the weighted data
            clf = DecisionStump()
            clf.fit(X, y, sample_weights=w)

            # 4. Calculate the weighted error (err) of the stump
            predictions = clf.predict(X)

            # Sum weights of misclassified samples
            # Add a small epsilon to prevent division by zero
            err = np.sum(w[predictions != y]) + 1e-10

            # 5. Calculate the learner's weight (alpha)
            # alpha = 0.5 * log((1 - err) / err)
            alpha = 0.5 * np.log((1.0 - err) / err)

            # 6. Update the sample weights
            # w = w * exp(-alpha * y * predictions)
            w *= np.exp(-alpha * y * predictions)

            # 7. Normalize the weights (so they sum to 1)
            w /= np.sum(w)

            # Store the trained learner and its alpha
            self.clfs.append(clf)
            self.alphas.append(alpha)

    def predict(self, X):
        """
        Makes a final prediction based on the weighted vote
        of all trained weak learners.
        """
        # Get predictions from all weak learners
        clf_preds = [alpha * clf.predict(X) for clf, alpha in zip(self.clfs, self.alphas)]

        # Sum the weighted predictions
        y_pred = np.sum(clf_preds, axis=0)

        # The final prediction is the sign of the sum
        return np.sign(y_pred)

In [ ]:
#-----------------------------------------------------------------
# 3. Example Usage
#-----------------------------------------------------------------

if __name__ == "__main__":

    print("Running AdaBoost from scratch...")

    # Generate a binary classification dataset
    X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_informative=2,
                               random_state=1, n_clusters_per_class=1)

    # AdaBoost algorithm requires labels to be -1 and 1
    y = np.where(y == 0, -1, 1)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- Train our custom AdaBoost ---
    # Using 5 weak learners as in the GFG article's example
    adaboost = AdaBoost(n_estimators=5)
    adaboost.fit(X_train, y_train)

    # Make predictions
    y_pred = adaboost.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Custom AdaBoost Accuracy: {accuracy:.4f}")


    # --- For comparison: Scikit-learn's AdaBoost ---
    from sklearn.ensemble import AdaBoostClassifier

    # We use DecisionTreeClassifier(max_depth=1) to mimic our DecisionStump
    # 'SAMME' is used for discrete boosting with -1/1 labels
    sklearn_ada = AdaBoostClassifier(
        n_estimators=5,
        algorithm='SAMME',
        random_state=42
    )

    sklearn_ada.fit(X_train, y_train)
    sklearn_pred = sklearn_ada.predict(X_test)

    sklearn_accuracy = accuracy_score(y_test, sklearn_pred)
    print(f"Scikit-learn AdaBoost Accuracy: {sklearn_accuracy:.4f}")

Running AdaBoost from scratch...
Custom AdaBoost Accuracy: 0.9000
Scikit-learn AdaBoost Accuracy: 0.9000


/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
